In [ ]:
import pandas as pd
def diff():
    df = pd.read_csv("backupData/sp500components.csv", index_col=0)
    df_last_components = df.iloc[-1, 0]
    df_last_components = df_last_components.split(",")
    df2 = pd.read_csv("data/sp500_companies.csv")
    df2_last_components = df2.iloc[:, 0].to_list()
    # calculate the difference between the two lists
    if sorted(list(df_last_components)) != sorted(list(set(df_last_components))) or sorted(list(df2_last_components)) != sorted(list(set(df2_last_components))):
        raise ValueError("The input lists must be sets")
    return (list(set(df_last_components) - set(df2_last_components)), list(set(df2_last_components) - set(df_last_components)))
print(diff())
def add_change():
    change = input("Enter a change in the format YYYY-MM-DD;Add1,Add2;Rem1,Rem2:")
    if change.count(";") != 2:
        print("The input must contain two semicolons")
        return
    change = change.split(";")
    change_date = change[0]
    add = change[1].split(",")
    remove = change[2].split(",")
    df = pd.read_csv("backupData/sp500components.csv", index_col=0)
    # last date in the dataframe
    last_date = pd.to_datetime(df.index[-1]).date()
    change_date = pd.to_datetime(change_date).date()
    # if last date is less than the change date
    if last_date < change_date:
        # get the last row in the dataframe
        last_row = df.iloc[-1, 0]
        last_row = last_row.split(",")
        # add the new components
        if add != [""]:
            last_row.extend(add)
        # remove the components
        for i in remove:
            if not i:
                continue
            if i not in last_row:
                raise ValueError("The component to be removed must be in the dataframe")
            last_row.remove(i)
        # remove duplicates
        last_row = sorted(list(set(last_row)))
        # convert the list to a string
        last_row = ",".join(last_row)
        # add the new row to the dataframe
        df.loc[change_date] = last_row
        # save the dataframe
        df.to_csv("backupData/sp500components.csv")

    else:
        raise ValueError("The date must be later than the last date in the dataframe")
add_change()

In [ ]:
import pandas as pd

df = pd.read_csv("backupData/sp500components.csv", index_col=0)
df.index = pd.to_datetime(df.index)
#df = df.resample("D").ffill()
#df = df.iloc[:,0].str.split(",", expand=True)
df

In [ ]:
# import os
# import utils
# data_dir = "test_data/"
# if not os.path.exists(data_dir):
#     os.mkdir(data_dir)

In [ ]:
# utils.fetch_yf_data('AAPL', data_dir, '2019-01-01', '2020-01-01')
# utils.fetch_yf_data('AAPL', data_dir, '2018-12-31', '2020-01-01')
# utils.fetch_yf_data('AAPL', data_dir, '2000-01-01')

# Test Metrics

In [ ]:
import os
from metrics import *
import plotting
data_dir = "test_data/"
if not os.path.exists(data_dir):
    os.mkdir(data_dir)
start_date = '2000-01-01'
end_date = '2024-01-01'

In [ ]:
def showMetricsCalculation(metric : Metric, y_axis = ["left", "right", "right"]):
    # plot all stages (data, processed, result)
    graph_list = [
        plotting.Graph(metric.data, y_axis=y_axis[0], df_color="black", df_label="data"), 
        plotting.Graph(metric.processed, y_axis=y_axis[1], df_color="blue", df_label="processed"), 
        plotting.Graph(metric.result, y_axis=y_axis[2], df_color="red", df_label="result")
    ]
    plotting.plot_graph(graph_list, neutral_line=None, title=metric.name)

In [ ]:
Metric.setPreferences(data_dir, start_date, end_date)
metrics = []
metrics.append(SP500Momentum())
for metric in metrics:
    metric.get()
    showMetricsCalculation(metric)